# Cheating

It occurred to me that the way I've designed the puzzle and solver classes opens the way to having a "cheating" solver. Basically a solver that replaces the puzzle with a pre-programmed sequence of numbers that obey the rules but do not match the original puzzle clues.

So just for fun let's see how easy that is to do and how I could improve the puzzle class to detect and block attempts to "cheat".

## Why was "Cheating" Possible?

First let's define what we mean by "cheating", and explain why it's *possible* in *this* module to "cheat" at Sudoku. Then we can look at specific cheats and what I would need to do to prevent them working.

By "cheating" I mean tricking the test harness into accepting a solution that is not in fact a proper solution to the puzzle it was asked to solved. It might be a solution to some other puzzle, but not the one given to the solver. Or we might be able to trick the test harness into accepting an answer that isn't even valid.

The root of the problem is that I am not storing the expected answers in the test data. I test only that a solution is *valid*, not that it is the *expected* solution for a given puzzle. Because I only test for validity, it's possible for a solver class to return *any valid puzzle* and my test harness would consider that "correct."

The rest of this notebook basically goes through the process I followed as I went "oh shit I've left a huge gap in my testing strategy", through to "huh, that's funny", and finally using this as an excuse to learn a bit more about Python.


## Modules required

We're using the [sudoku](../puzzle/sudoku.py) and [tester](../puzzle/tester.py) modules used elsewhere, as well as a small number of standard libraries. We have to make a slight adjustment to the notebook's environment in order to find these modules, since this notebook is in a sub-directory.

In [2]:
import copy
import sys

sys.path.insert(-1, '..')
import puzzle.tester as tester
import puzzle.sudoku as su
from puzzle.jupyter_helpers import *
display(HTML(SUDOKU_CSS))

The "purpose" of cheating is to beat the performance of legitimate solvers. We'll be using Pandas and Matplotlib then to assess results.

In [3]:
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams["figure.figsize"] = [12, 6]
pd.set_option('precision', 3)

Finally, some functions to standardise how we show test results.

In [4]:
def format_if_num(num):
    if num is None:
        return num
    else:
        return f"{num:.3f}"

def show_results(pt):
    solver_labels = list(pt.get_solver_labels())
    df = pd.DataFrame(pt.get_test_results())
    return df.style.highlight_null().\
        highlight_max(axis=1, color='darkorange', subset=solver_labels).\
        highlight_min(axis=1, color='green', subset=solver_labels).\
        format({m: format_if_num for m in solver_labels})

---
# Cheating Attempts

## First attempt: Lie

First attempt was based on this code in the original implementation of `is_solved` (if you check [the code](../puzzle/sudoku.py) you'll see the method is no longer implemented this way - see below for why):

```python
def is_solved(self):
    return self.is_puzzle_valid() and self._num_empty_cells == 0
```

So, how about a solver that just plain lies by replacing the number of empty cells left? To try this I've created a new class `CheatingSolver` which tries to replace `puzzle`'s private attribute tracking the number of empty cells and then just *always* return `True`.


In [5]:
class CheatingSolver:
    def solve(self, puzzle):
        """Easiest way to cheat would be to trick the is_solved() method on the puzzle to always returning True"""
        puzzle._num_empty_cells = 0
        return True

puzzle = su.SudokuPuzzle(starting_grid=su.from_string(su.SAMPLE_PUZZLES[0]['puzzle']))
solver = CheatingSolver()
solver.solve(puzzle)

True

So the solver will always return `True` but the puzzle itself should know that it's not really solved. I changed the `is_solved` method to actually check that every cell has a value.

```python
def is_solved(self):
    if self.is_puzzle_valid():
        for i in range(self.max_value()):
            for j in range(self.max_value()):
                if self.is_empty(i, j):
                    return False
        return True
    else:
        return False
```

In [6]:
puzzle.is_solved()

False

Now if we use this in the `PuzzleTester` then we want to make sure that it's detecting that the puzzle isn't really solved. You'll notice there's an initialization parameter, `anti_cheat_check` that has been set to `False`. We'll demonstrate later why I added that in.

In [7]:
include_levels = ['Kids', 'Easy', 'Moderate', 'Hard']  # , 'Diabolical', 'Pathalogical']
test_cases = [x for x in su.SAMPLE_PUZZLES if x['level'] in include_levels]
pt = tester.PuzzleTester(puzzle_class=su.SudokuPuzzle, anti_cheat_check=False)
pt.add_test_cases(test_cases)

8

For our anti-cheat check to be "working" we want to see that `CheatingSolver` has no results recorded for it.

In [8]:
solver = CheatingSolver()
pt.run_tests(solver)
show_results(pt)

,label,level,starting_clues,CheatingSolver
0,SMH 1,Kids,31,None
1,SMH 2,Easy,24,None
2,KTH 1,Easy,30,None
3,Rico Alan Heart,Easy,22,None
4,SMH 3,Moderate,26,None
5,SMH 4,Hard,22,None
6,SMH 5,Hard,25,None
7,Greg [2017],Hard,21,None


I had to change `PuzzleTester` class to check the return value of the puzzle's `is_solved` method, rather than trust the solver's return value from `solve`. If the puzzle asserts that it is NOT solved then no result is recorded for the solver.

I also changed the `num_empty_cells` attribute in `SudokuPuzzle` from [protected to private](https://www.tutorialsteacher.com/python/private-and-protected-access-modifiers-in-python), so the attempt to replace the real value no longer works.

In [9]:
puzzle.num_empty_cells()

50

# Second attempt: Replace puzzle with a canned solution

Since our really simple cheater no longer worked I needed a more sophisticated version. We could just fill in the blank cells with "1" (or random values) but then the `is_puzzle_valid` check would fail, at which point we may as well solve it properly. 

So maybe what our cheat needs to do is fill in *all* cells in a rule-abiding way. We won't be actually solving the original puzzle. Basically, we're just writing a "pre-solved" puzzle over the top.

In [10]:
class CheatingSolver:
    def solve(self, puzzle):
        """Write a pre-solved puzzle in over the top of the provided one"""
        starting_values = [0, 3, 6, 1, 4, 7, 2, 5, 8]
        max_value = puzzle.max_value
        assert max_value == 9, "I can't handle puzzles other than 9x9"
        puzzle.clear_all()
        for i in range(max_value):
            for j in range(max_value):
                puzzle.set(i, j, (starting_values[i] + j) % max_value + 1)
        return True

In [11]:
puzzle = su.SudokuPuzzle(starting_grid=su.from_string(su.SAMPLE_PUZZLES[0]['puzzle']))
solver = CheatingSolver()
solver.solve(puzzle)
puzzle.is_solved()

True

So this cheat works, at least as far as the `puzzle` instance is concerned.

Now, the whole point of cheating here is to be faster than a real solver, so let's test performance.


In [12]:
# Runs all the "legit" solvers
for m in su.SOLVERS:
    solver = su.SudokuSolver(method=m)
    pt.run_tests(solver, m)

In [13]:
# Runs the "cheat" solver
solver = CheatingSolver()
pt.run_tests(solver)
solver_labels = list(pt.get_solver_labels())

In [14]:
show_results(pt)

,label,level,starting_clues,CheatingSolver,backtracking,constraintpropogation,deductive,sat
0,SMH 1,Kids,31,0.002,0.004,0.002,0.002,0.022
1,SMH 2,Easy,24,0.002,0.190,0.002,0.003,0.020
2,KTH 1,Easy,30,0.001,0.011,0.001,0.002,0.016
3,Rico Alan Heart,Easy,22,0.001,0.070,0.021,0.007,0.017
4,SMH 3,Moderate,26,0.001,0.075,0.020,0.027,0.017
5,SMH 4,Hard,22,0.001,1.355,0.024,0.016,0.018
6,SMH 5,Hard,25,0.001,0.568,0.024,0.013,0.020
7,Greg [2017],Hard,21,0.001,0.560,0.037,0.041,0.020


Orange in each row is the slowest time, and green in each row is the fastest. Oddly, the cheating solver doesn't *always* win, but it wins enough that there appears a sufficient motivation to cheat. So let's fix that.


---
# Catching Cheats

In the absence of prepared solutions to compare against, one way to prevent the new cheat is to compare the "solved" puzzle with a copy of the original. That way we can detect that the starting clues have been replaced.

We can't do this in the `SudokuPuzzle` itself. Python's private attributes can be tampered with (it's [more a naming convention](https://docs.python.org/3/tutorial/classes.html#tut-private) to stop programmers shooting themselves in the foot than a security control). Since we're trying to guard against cheating we can assume an attacker will happily ignore convention.

If we assume that the caller (test harness) can be trusted then we can let the caller verify that the original puzzle is OK. We'll just need a function that confirms if the starting clues in one puzzle also exist in the second.


In [15]:
def has_same_clues(a, b):
    """Returns true if the non empty cells in a have the same value in b"""
    if a.max_value != b.max_value:
        return False
    
    for i in range(a.max_value):
        for j in range(a.max_value):
            if not a.is_empty(i, j) and a.get(i, j) != b.get(i, j):
                return False
    return True

In [16]:
puzzle = su.SudokuPuzzle(starting_grid=su.from_string(su.SAMPLE_PUZZLES[-1]['puzzle']))
original = copy.deepcopy(puzzle)
has_same_clues(original, puzzle)

True

In [17]:
# Solver is cheating and will replace puzzle
solver.solve(puzzle)
puzzle.is_solved()

True

In [18]:
# Should return False because puzzle has been replaced
has_same_clues(original, puzzle)

False

This is the fix I put in the `PuzzleTester` class. The `anti_cheat_check` is on by default. The `run_single_test` method makes a copy of each puzzle *before* calling the solver, then compares the "solved" puzzle to the copy. If the clues in the original aren't present in the solved puzzle then it's not a real solution.

So now we can re-run the tests and check to see that our cheater won't prosper.


In [19]:
# New instance of PuzzleTester, anti_cheat_check is True by default
pt = tester.PuzzleTester(puzzle_class=su.SudokuPuzzle)
pt.add_test_cases(test_cases)

8

In [20]:
# Runs the "cheat" solver
solver = CheatingSolver()
pt.run_tests(solver)

8

In [21]:
# Runs all the "legit" solvers
for m in su.SOLVERS:
    solver = su.SudokuSolver(method=m)
    pt.run_tests(solver, m)

In [22]:
show_results(pt)

,label,level,starting_clues,CheatingSolver,backtracking,constraintpropogation,deductive,sat
0,SMH 1,Kids,31,None,0.003,0.002,0.002,0.021
1,SMH 2,Easy,24,None,0.199,0.002,0.003,0.017
2,KTH 1,Easy,30,None,0.011,0.001,0.002,0.017
3,Rico Alan Heart,Easy,22,None,0.068,0.021,0.007,0.017
4,SMH 3,Moderate,26,None,0.073,0.019,0.027,0.017
5,SMH 4,Hard,22,None,1.344,0.024,0.023,0.017
6,SMH 5,Hard,25,None,0.557,0.024,0.019,0.018
7,Greg [2017],Hard,21,None,0.543,0.037,0.046,0.019


OK! Our cheating solver has had no results recorded for it, because the answer it gives does not match the starting clues!

---
# Next Steps and Rabbit Holes

There are probably ways to defeat these checks, particularly in a language like Python where "[monkey patching](https://medium.com/@chipiga86/python-monkey-patching-like-a-boss-87d7ddb8098e)" is a thing and everything is dynamic. For example:

* Could we subclass `SudokuPuzzle` and modify the methods there?
* The `PuzzleTester` method `run_tests` takes a `callback` parameter that's called just before each test is run, and then finally when all tests are complete. Could we hijack that to fake our test results? You'd have to do it from inside the solver class for it to be a real "cheat"...
* Can the solver class access and modify the copy of the original puzzle?

These might be a fun way to learn more about the internals of Python, but for now I'm declaring this "done" and moving on to the next puzzle...

## Diversion #1: LeetCode

I came across [LeetCode's Sudoku Solver](https://leetcode.com/problems/sudoku-solver/) and attempted my "canned solution cheat" there. It doesn't work because LeetCode is sensible enough to actually have an *expected* output and compares against that. I don't think it even checks if the solution is valid as such, only that it matches the expected output for the test case. I tried this "solution":

```python
class Solution:
    def solveSudoku(self, board: List[List[str]]) -> None:
        """
        Do not return anything, modify board in-place instead.
        """
        expected = [["5","3","4","6","7","8","9","1","2"],
                    ["6","7","2","1","9","5","3","4","8"],
                    ["1","9","8","3","4","2","5","6","7"],
                    ["8","5","9","7","6","1","4","2","3"],
                    ["4","2","6","8","5","3","7","9","1"],
                    ["7","1","3","9","2","4","8","5","6"],
                    ["9","6","1","5","3","7","2","8","4"],
                    ["2","8","7","4","1","9","6","3","5"],
                    ["3","4","5","2","8","6","1","7","9"]]
        max_value = len(expected)
        for i in range(max_value):
            for j in range(max_value):
                board[i][j] = expected[i][j]
```

You can run this answer it will be accepted. If you submit this answer it's run using multiple test cases, so will fail on the second case. I tried up to 3 canned solutions before I got bored. In any event, on LeetCode you're supposed to submit your answer for peer review, so this approach isn't really going to help much. :-)

The LeetCode testing mechanism appears to be pretty robust. I used `inspect` to dump the test driver's source code to `stdout` (which LeetCode let's you examine - I guess to help debug your code). The tested code is run in its own process, and its output carefully serialised (but not evaluated) into an output file. A separate process then examines the submitted output and compares to the expected/acceptable output. Because it's in a separate process there's no (easy) way of finding out the answer in advance. I guess they have to do it this way because you can code your answer in lots of different languages.

I wondered about using `subprocess` to learn more about the test environment but didn't try it. Although the [terms of service](https://leetcode.com/terms/) don't mention it I assume the site owners would rather not have people poking around like that. I suspect the test code is run in a Docker container that does not have access to the expected answers file.

## Postscript 2: Mucking about with PuzzleTester's internals

So begins a little rabbit hole I went down to learn more about Python's internals and the `inspect` module.

Since we know how Python munges private attribute names, can we just turn off the anti-cheat?

In [23]:
pt._PuzzleTester__anti_cheat_check = False
solver = CheatingSolver()
pt.run_tests(solver)
show_results(pt)

,label,level,starting_clues,CheatingSolver,backtracking,constraintpropogation,deductive,sat
0,SMH 1,Kids,31,0.001,0.003,0.002,0.002,0.021
1,SMH 2,Easy,24,0.001,0.199,0.002,0.003,0.017
2,KTH 1,Easy,30,0.001,0.011,0.001,0.002,0.017
3,Rico Alan Heart,Easy,22,0.001,0.068,0.021,0.007,0.017
4,SMH 3,Moderate,26,0.001,0.073,0.019,0.027,0.017
5,SMH 4,Hard,22,0.001,1.344,0.024,0.023,0.017
6,SMH 5,Hard,25,0.002,0.557,0.024,0.019,0.018
7,Greg [2017],Hard,21,0.001,0.543,0.037,0.046,0.019


Yeah OK - but that's the caller turning it off not the solver. How could the solver do it? And is there a way for Python to stop us doing that? (A: It [appears not](https://stackoverflow.com/questions/10929004/how-to-restrict-setting-an-attribute-outside-of-constructor))

Can the solver find out much about its environment when the `solve` method is invoked?

First, let's undo that hack to the private attribute, and rerun the tests so that `CheatingSolver` can be seen to be detected.

In [24]:
pt._PuzzleTester__anti_cheat_check = True
solver = CheatingSolver()
pt.run_tests(solver)
show_results(pt)

,label,level,starting_clues,CheatingSolver,backtracking,constraintpropogation,deductive,sat
0,SMH 1,Kids,31,None,0.003,0.002,0.002,0.021
1,SMH 2,Easy,24,None,0.199,0.002,0.003,0.017
2,KTH 1,Easy,30,None,0.011,0.001,0.002,0.017
3,Rico Alan Heart,Easy,22,None,0.068,0.021,0.007,0.017
4,SMH 3,Moderate,26,None,0.073,0.019,0.027,0.017
5,SMH 4,Hard,22,None,1.344,0.024,0.023,0.017
6,SMH 5,Hard,25,None,0.557,0.024,0.019,0.018
7,Greg [2017],Hard,21,None,0.543,0.037,0.046,0.019


Now, let's try and re-implement the hack, this time inside the solve method...

In [25]:
class SneakySolver(CheatingSolver):
    def solve(self, puzzle):
        # Find the PuzzleTester instance -- assume we don't know
        # the variable name already
        for x in globals():
            if isinstance(globals()[x], tester.PuzzleTester):
                mypt = globals()[x]
                if mypt._PuzzleTester__anti_cheat_check:
                    mypt._PuzzleTester__anti_cheat_check = False
                    print(f"Have DISABLED anti-cheat check!")
                    
        # Now solve the puzzle in the usual "cheating" way
        return super().solve(puzzle)

In [26]:
solver = SneakySolver()
pt.run_tests(solver)
show_results(pt)

Have DISABLED anti-cheat check!


,label,level,starting_clues,CheatingSolver,backtracking,constraintpropogation,deductive,sat,SneakySolver
0,SMH 1,Kids,31,None,0.003,0.002,0.002,0.021,0.001
1,SMH 2,Easy,24,None,0.199,0.002,0.003,0.017,0.001
2,KTH 1,Easy,30,None,0.011,0.001,0.002,0.017,0.001
3,Rico Alan Heart,Easy,22,None,0.068,0.021,0.007,0.017,0.001
4,SMH 3,Moderate,26,None,0.073,0.019,0.027,0.017,0.001
5,SMH 4,Hard,22,None,1.344,0.024,0.023,0.017,0.001
6,SMH 5,Hard,25,None,0.557,0.024,0.019,0.018,0.001
7,Greg [2017],Hard,21,None,0.543,0.037,0.046,0.019,0.001


OK, that "worked." But I could always take out the ability to turn "anti-cheating" off -- or report to the caller that it has been turned off. Also, the cheat required knowledge of the internal workings of `PuzzleTester`.

So let's assume that I took out the code that makes anti-cheat checking conditional.

In [27]:
pt._PuzzleTester__anti_cheat_check = True

This new `BetterPuzzleTester` class overrides `PuzzleTester`'s `run_single_test` method to *always* check the solution against the original puzzle -- so disabling the `anti_cheat_check` attribute will have no effect.

In [28]:
class BetterPuzzleTester(tester.PuzzleTester):
    def run_single_test(self, test_puzzle, solver):
        puz = self.puzzle_class(starting_grid=su.from_string(test_puzzle))
        orig = copy.deepcopy(puz)
        claimed_solved = solver.solve(puz)

        # Almost the same as original - except for extra check
        if claimed_solved and has_same_clues(orig, puz):
            self.__last_was_solved = puz.is_solved()
        else:
            self.__last_was_solved = False
        return self.__last_was_solved

In [29]:
# New instance of PuzzleTester, anti_cheat_check is True by default
newpt = BetterPuzzleTester(puzzle_class=su.SudokuPuzzle)
newpt.add_test_cases(test_cases)
newpt.run_tests(solver)
show_results(newpt)

Have DISABLED anti-cheat check!
Have DISABLED anti-cheat check!


,label,level,starting_clues,SneakySolver
0,SMH 1,Kids,31,None
1,SMH 2,Easy,24,None
2,KTH 1,Easy,30,None
3,Rico Alan Heart,Easy,22,None
4,SMH 3,Moderate,26,None
5,SMH 4,Hard,22,None
6,SMH 5,Hard,25,None
7,Greg [2017],Hard,21,None


## Postscript 3: Monkey Patching  <a class="anchor" id="monkey_patching"></a>

Let's try another way.

The method `run_single_test` calls the function `has_same_clues` defined in [tester.py](../puzzle/tester.py). If this ever returns `False` after running the solver, then the cheat is detected.

So, could our cheating solver monkey patch that function to always return `True`?

Turns out, "yes" and it's easy.

In [30]:
# Preserving the original function definition
original_has_same_clues = tester.has_same_clues

class SneakySolver(CheatingSolver):
    def solve(self, puzzle):
        # Monkey patch has_same_clues to always return True
        tester.has_same_clues = lambda x, y : True
        return super().solve(puzzle)
    

In [31]:
newpt = tester.PuzzleTester(puzzle_class=su.SudokuPuzzle)
newpt.add_test_cases(test_cases)

solver = SneakySolver()
newpt.run_tests(solver)
show_results(newpt)

,label,level,starting_clues,SneakySolver
0,SMH 1,Kids,31,0.002
1,SMH 2,Easy,24,0.003
2,KTH 1,Easy,30,0.003
3,Rico Alan Heart,Easy,22,0.002
4,SMH 3,Moderate,26,0.002
5,SMH 4,Hard,22,0.001
6,SMH 5,Hard,25,0.003
7,Greg [2017],Hard,21,0.002


To detect this, we could run a little test in `run_single_test`, because if `has_same_clues` returns True for `(original, puzzle)` then it should return `False` for `(puzzle, original)`. 

In [32]:
class BetterPuzzleTester(tester.PuzzleTester):
    def run_single_test(self, test_puzzle, solver):
        puz = self.puzzle_class(starting_grid=su.from_string(test_puzzle))
        orig = copy.deepcopy(puz)
        claimed_solved = solver.solve(puz)

        # Almost the same as original - except for extra check
        if claimed_solved and has_same_clues(orig, puz) and not has_same_clues(puz, orig):
            self.__last_was_solved = puz.is_solved()
        else:
            self.__last_was_solved = False
        return self.__last_was_solved

In [33]:
newpt = BetterPuzzleTester(puzzle_class=su.SudokuPuzzle)
newpt.add_test_cases(test_cases)

8

In [34]:
solver = SneakySolver()
newpt.run_tests(solver)
show_results(newpt)

,label,level,starting_clues,SneakySolver
0,SMH 1,Kids,31,None
1,SMH 2,Easy,24,None
2,KTH 1,Easy,30,None
3,Rico Alan Heart,Easy,22,None
4,SMH 3,Moderate,26,None
5,SMH 4,Hard,22,None
6,SMH 5,Hard,25,None
7,Greg [2017],Hard,21,None


This might be an acceptable approach to cheat detection. I thought of checking for modification or monkey patching but it seems that even those checks could be themselves patched, so instead we run "tests" to check that the function *behaves* as expected.

In [35]:
# Restore the original `has_same_clues`
tester.has_same_clues = original_has_same_clues

## Postscript 4: Mess with the caller's frame

Last one. Can we use `inspect` to find the calling method's copy of the puzzle, and change it so that our faked solution *looks* legitimate, because it matches the "copy"?

In [38]:
import inspect

class SneakySolver(CheatingSolver):
    def solve(self, puzzle):
        # We know we can find the PuzzleTester instance from previous
        # example. Take a shortcut this time since we know the local 
        # variable name (pt)
        mypt = globals()['pt']
        super().solve(puzzle)  # Fake solver, just replaces puzzle

        # PuzzleTester has made a copy of the puzzle in orig        
        me = inspect.currentframe()
        caller = inspect.getouterframes(me)[1][0]
        orig = caller.f_locals['orig']
        
        # Tried just repointing f_locals['orig'] but it doesn't work
        # Need to update it instead. It's a SudokuPuzzle instance so it's easy
        orig.clear_all()
        for x in range(orig.max_value - 1):
            for y in range(orig.max_value):
                orig.set(x, y, puzzle.get(x, y))
        return True

In [39]:
solver = SneakySolver()
pt.run_tests(solver)
show_results(pt)

,label,level,starting_clues,CheatingSolver,backtracking,constraintpropogation,deductive,sat,SneakySolver
0,SMH 1,Kids,31,None,0.003,0.002,0.002,0.021,0.018
1,SMH 2,Easy,24,None,0.199,0.002,0.003,0.017,0.028
2,KTH 1,Easy,30,None,0.011,0.001,0.002,0.017,0.029
3,Rico Alan Heart,Easy,22,None,0.068,0.021,0.007,0.017,0.038
4,SMH 3,Moderate,26,None,0.073,0.019,0.027,0.017,0.017
5,SMH 4,Hard,22,None,1.344,0.024,0.023,0.017,0.012
6,SMH 5,Hard,25,None,0.557,0.024,0.019,0.018,0.010
7,Greg [2017],Hard,21,None,0.543,0.037,0.046,0.019,0.010


I kind of like this cheat. It's using the `inspect` module to access the calling function's frame, finds the variable being used to keep a copy of the original puzzle, modifies that copy (but leaves some cells unsolved), and then returns the faked solution, which just so happens to match the callers copy!

Something odd though -- it's not working on the "new" tester class:

In [40]:
newpt.run_tests(solver)
show_results(newpt)

,label,level,starting_clues,SneakySolver
0,SMH 1,Kids,31,None
1,SMH 2,Easy,24,None
2,KTH 1,Easy,30,None
3,Rico Alan Heart,Easy,22,None
4,SMH 3,Moderate,26,None
5,SMH 4,Hard,22,None
6,SMH 5,Hard,25,None
7,Greg [2017],Hard,21,None


Worked out why: It's because `__last_was_solved` is "private" and I only over-rode `run_single_test` which sets it. But when `run_tests` runs and checks `__last_was_solved` it's using the parent class version (name munged to `_PuzzleTester__last_was_solved` and not the child class version (name munged to `_BetterPuzzleTester__last_was_solved`.

Me: Stupid...stupid...stupid...

In [41]:
class BetterPuzzleTester(tester.PuzzleTester):
    def run_single_test(self, test_puzzle, solver):
        puz = self.puzzle_class(starting_grid=su.from_string(test_puzzle))
        orig = copy.deepcopy(puz)
        claimed_solved = solver.solve(puz)

        # Almost the same as original - except for extra check
        if claimed_solved and has_same_clues(orig, puz) and not has_same_clues(puz, orig):
            print(f"claimed={claimed_solved}; check1={has_same_clues(orig, puz)}; check2={not has_same_clues(puz, orig)}; puz={puz.is_solved()}")
            self._PuzzleTester__last_was_solved = puz.is_solved()
        else:
            print(f"claimed={claimed_solved}; check1={has_same_clues(orig, puz)}; check2={not has_same_clues(puz, orig)}")
            self._PuzzleTester__last_was_solved = False
        return self._PuzzleTester__last_was_solved

In [42]:
newpt = BetterPuzzleTester(puzzle_class=su.SudokuPuzzle)
newpt.add_test_cases(test_cases)
newpt.run_tests(solver)

claimed=True; check1=True; check2=True; puz=True
claimed=True; check1=True; check2=True; puz=True
claimed=True; check1=True; check2=True; puz=True
claimed=True; check1=True; check2=True; puz=True
claimed=True; check1=True; check2=True; puz=True
claimed=True; check1=True; check2=True; puz=True
claimed=True; check1=True; check2=True; puz=True
claimed=True; check1=True; check2=True; puz=True


8

In [43]:
show_results(newpt)

,label,level,starting_clues,SneakySolver
0,SMH 1,Kids,31,0.019
1,SMH 2,Easy,24,0.016
2,KTH 1,Easy,30,0.023
3,Rico Alan Heart,Easy,22,0.029
4,SMH 3,Moderate,26,0.022
5,SMH 4,Hard,22,0.045
6,SMH 5,Hard,25,0.018
7,Greg [2017],Hard,21,0.020


Passing back the result of `run_single_test` as a class attribute felt dodgy at the time...and now I know why. :-)

In [72]:
class Banana:
    def stack_dump(self):
        # Get my frame, then walk up the stack
        me = inspect.currentframe()
        caller = inspect.getouterframes(me)
        
        for i, x in enumerate(caller):
            # FrameInfo(frame, filename, lineno, function, code_context, index) is returned.
            print(f"Frame #{i} is in {x.function} @ {x.code_context} defined by {x.filename}:{x.lineno}")
            halfpt = len(inspect.getsource(x[0])) // 2
            print(inspect.getsource(x[0])[0:halfpt])

b = Banana()
b.stack_dump()

Frame #0 is in stack_dump @ ['        caller = inspect.getouterframes(me)\n'] defined by <ipython-input-72-b8067fc6aeab>:5
    def stack_dump(self):
        # Get my frame, then walk up the stack
        me = inspect.currentframe()
        caller = inspect.getouterframes(me)
        
        for i, x in enumerate(caller):
            # FrameInfo(frame, filename, lineno, func
Frame #1 is in <module> @ ['b.stack_dump()\n'] defined by <ipython-input-72-b8067fc6aeab>:14
class Banana:
    def stack_dump(self):
        # Get my frame, then walk up the stack
        me = inspect.currentframe()
        caller = inspect.getouterframes(me)
        
        for i, x in enumerate(caller):
            # FrameInfo(frame, filename, lineno, function, c
Frame #2 is in run_code @ ['                    exec(code_obj, self.user_global_ns, self.user_ns)\n'] defined by /anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3417
    async def run_code(self, code_obj, result=None, *, async_=F